In [31]:
import pandas as pd
import glob
import scipy.io as io
import numpy
from tfcat import TFCat
from shapely.geometry import MultiPoint, Point, Polygon
from astropy.time import Time
import matplotlib.pyplot as pyplot
import matplotlib.colors as colors
import h5py
import numpy as np
from datetime import datetime
from os import path
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
import seaborn as sns 
from scipy.stats import binned_statistic_2d
from tfcat_tools.mask import Mask

plt.rcParams["figure.figsize"] = (200,100)


/Users/clouis/opt/anaconda3/lib/python3.8/site-packages/spacepy/time.py:2294: UserWarning: Leapseconds may be out of date. Use spacepy.toolbox.update(leapsecs=True)
  warnings.warn('Leapseconds may be out of date.'


In [2]:
def get_polygons(polygon_fp,start, end, type_):
    date_time = [start, end]
    #Convert start/stop times to unix from isoformat.
    unix_start, unix_end = Time(date_time,format='isot').to_value('unix')
    
    #array of polygons found within time interval specified.
    polygon_array=[]
    if path.exists(polygon_fp):
        catalogue = TFCat.from_file(polygon_fp)
        for i in range(len(catalogue)):
                
                time_points=np.array(catalogue._data.features[i]['geometry']['coordinates'][0])[:,0]
                label=catalogue._data.features[i]['properties']['feature_type']
                if any(time_points >= unix_start) and any(time_points <= unix_end):
                    if label in type_:
                        polygon_array.append(catalogue._data.features[i]['geometry']['coordinates'][0])
    #polgyon array contains a list of the co-ordinates for each polygon within the time interval           
    return polygon_array
                   

In [3]:
#Polar ephemeris data taken from https://sscweb.gsfc.nasa.gov/cgi-bin/Locator.cgi
df = pd.read_csv("/Users/clouis/Documents/Data/POLAR/data/polar_geo_ephemeris.txt", delimiter = " ", header = None,
                    names=["Date", "Time", "GEO_LAT", "GEO_LONG", "Distance_R_E"])
df["Date"] = "19"+df["Date"]                  

#df = pd.read_csv("polar_ephemeris.txt", delimiter = " ", header = None, names = ["Year", 
#                                                                                     "Time", 
#                                                                                     "GSE_LAT",
#                                                                                     "GSE_LONG",
#                                                                                     "Distance_R_E"])

In [4]:
df.index = pd.to_datetime(df["Date"]+"T"+df["Time"])
df.drop(["Date", "Time"], axis = 1, inplace = True)


In [6]:

time_view_start = '1996-03-25'
time_view_end ='1996-05-03'
val='Flux'
file_data=glob.glob("/Users/clouis/Documents/Data/POLAR/data/po_h1_pwi.preprocessed.hdf5")[0]
polygon_fp=glob.glob("/Users/clouis/Documents/Data/POLAR/data/catalogue_Polar.json")[0]
#type_=['AKR','AKR1','AKR2']
#type_=['HAKR','HAKR1','HAKR2']
#type_=['LFE','LFE1','LFE2']
#type_=['FCE','FCE1','FCE2']
type_=['AKR','AKR1','AKR2','HAKR','HAKR1','HAKR2','FCE','FCE1','FCE2']


In [7]:
#Read in data from file.
file = h5py.File(file_data, 'r')

cols = list(file.keys())


In [8]:

flux = pd.DataFrame(file.get(cols[1])[:])
flux.index = Time(file.get(cols[3])[:], format = "jd").datetime64
flux.columns = file.get(cols[2])[:]

#flux

In [9]:
#Restricting flux and time data to be within the user defined start/end times
flux = flux[(flux.index > time_view_start) & (flux.index < time_view_end)]
df = df[(df.index > time_view_start) & (df.index < time_view_end)]

In [10]:
#Interpolating the ephemeris data so it lines up with the flux data
pos_interp = pd.DataFrame(index = flux.index)
for column in df.columns: 
    pos_interp[column] = np.interp(flux.index, df.index, df[column])

In [11]:
coords_index = ["GEO_LAT", "GEO_LONG", "Distance_R_E"]

In [12]:
coord_geo = numpy.array(pos_interp)
time_coords_geo = pos_interp.index
frequencies = flux.columns.to_numpy().astype("float64")
flux = numpy.array(flux)

array([ 13008.68788311,  13889.76981329,  14830.52765964,  15835.00328803,
        16907.51232098,  18052.66267928,  19275.37437943,  20580.900672  ,
        21974.85061159,  23463.21315563,  25052.3828952 ,  26749.18752878,
        28560.91719671,  30495.35580247,  32560.81445544,  34766.16717866,
        37120.88903516,  39635.09683651,  42319.59260867,  45185.91000172,
        48246.36384295,  51514.10304623,  55003.1671049 ,  58728.54641097,
        62706.24665971,  66953.35761645,  71488.12654094,  76330.03658472,
        81499.89049843,  87019.90000861,  92913.78124802,  99206.85664947,
       105926.16374094, 113100.57130948, 120760.90343284, 128940.07191186,
       137673.21767247, 146997.86174502, 156954.06646931, 167584.60761818,
       178935.15817911, 191054.48458329, 203994.6562254 , 217811.26917425,
       232563.68503535, 248315.28599171, 265133.74711866, 283091.32714257,
       302265.1788928 , 322737.68078062, 344596.79072921, 367936.42407559,
       392856.85706782, 4

In [ ]:
#flux.index = Time(flux.index).unix
#pos_interp.index = Time(pos_interp.index).unix


In [23]:
#Importing polygons 
polygon_array=get_polygons(polygon_fp, time_view_start, time_view_end, type_)
#polygon_array

In [24]:
polygon_array

[[[827733788.45277, 497005.22717],
  [827733235.124289, 623269.229541],
  [827732128.467326, 628239.626425],
  [827731200.968998, 756571.338686],
  [827730275.609804, 756571.338686],
  [827729719.861038, 506973.262543],
  [827729524.568635, 724801.093665],
  [827729133.98383, 517141.218811],
  [827728775.947751, 633249.660826],
  [827728352.814218, 533835.822452],
  [827728124.973048, 643389.909097],
  [827727994.77814, 521265.274645],
  [827727441.449659, 680176.656148],
  [827727082.073364, 756571.338686],
  [827727040.492213, 756571.338686],
  [827726855.57245, 638299.648844],
  [827726461.204173, 756571.338686],
  [827726324.045501, 756571.338686],
  [827726074.402839, 594261.847246],
  [827725097.940825, 596626.677477],
  [827724056.381316, 535960.190854],
  [827723470.504108, 497005.22717],
  [827723079.919302, 557674.429121],
  [827722494.042094, 483378.375226],
  [827721517.58008, 548885.11321],
  [827721126.995274, 481462.423697],
  [827720280.728209, 483378.375226],
  [827719

In [ ]:
##combining flux and interpolated ephemeris
#flux = pd.concat([flux, pos_interp], axis = 1)

##coords = ["GSE_LAT", "GSE_LONG", "Distance_R_E"]
#coords = ["GEO_LAT", "GEO_LONG", "Distance_R_E"]

#del pos_interp
#del df

In [ ]:
#defining latitude and frequency coordinates for map
#latitudes = np.arange(-90,90.1,0.1)
latitudes = numpy.linspace(-90,90,181)



In [ ]:
distribution = numpy.zeros(shape=(frequencies.shape[0], len(latitudes)-1))

In [ ]:
for i in range(len(frequencies)):
    occurrence_mask = numpy.where(flux[frequencies[i]] > 0, numpy.ones_like(flux[:,i], dtype=bool), False)
    distribution[i, :] = numpy.histogram(flux["GEO_LAT"]*occurrence_mask, bins=latitudes)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
polygon_point_dict = {"GEO_LAT": [], 
                      "GEO_LONG": [], 
                      "Distance_R_E": [], 
                      "Time": [], 
                      "Frequency": [], 
                      "Flux": [], 
                      "Polygon": []}


#For each polygon a rectangle is defined using their min frequency, max frequency, start time, and end time. 
#The intersection of the polygon and this rectangle are found and the time, frequency, flux, and ephemeris for each
#point within  each polygon are saved to a dictionary  
pg = 0
for polygon in tqdm(polygon_array): 
    new_pg = False
    Min_time = polygon[0][0]
    Max_time = polygon[0][0]
    Min_freq = polygon[0][1]
    Max_freq = polygon[0][1]
    for point in polygon:
        if point[0] < Min_time: 
            Min_time = point[0]
        elif point[0] > Max_time:
            Max_time = point[0]
        if point[1] < Min_freq: 
            Min_freq = point[1]
        elif point[1] > Max_freq:
            Max = point[1]
    time_mask = (flux.drop(coords, axis = 1).index < Max_time) & (flux.drop(coords, axis = 1).index > Min_time)
    freq_mask = (flux.drop(coords, axis = 1).columns < Max_freq) & (flux.drop(coords, axis = 1).columns > Min_freq)
    reduced = flux.drop(coords, axis = 1)[flux.drop(coords, axis = 1).columns[freq_mask]][time_mask]
    reduced_frequencies = np.empty(shape = np.shape(reduced)[::-1])
    
    mp_list = []
    for red_time in reduced.index: 
        for red_freq in reduced.columns:
            mp_list.append(Point(red_time, red_freq))
    mp_list = MultiPoint(mp_list)
    i = 0
    
    for frequency in reduced.columns.to_numpy(): 
        reduced_frequencies[i] = frequency
        i += 1 
    poly_1 = np.array(Polygon(polygon).intersection(mp_list))
    poly_1 = poly_1.tolist()
    for point in range(len(poly_1)):
        try:
            polygon_point_dict["Time"].append(poly_1[point][0])
            polygon_point_dict["Frequency"].append( poly_1[point][1])
            polygon_point_dict["Flux"].append( reduced[reduced.columns[reduced.columns == poly_1[point][1]]][reduced.index == poly_1[point][0]].values[0][0])
            polygon_point_dict["GEO_LAT"].append(flux[coords][flux.index == poly_1[point][0]].values[0][0])
            polygon_point_dict["GEO_LONG"].append(flux[coords][flux.index == poly_1[point][0]].values[0][1])
            polygon_point_dict["Distance_R_E"].append(flux[coords][flux.index == poly_1[point][0]].values[0][2])
            polygon_point_dict["Polygon"].append(pg)        
            new_pg = True
        except: 
            print(poly_1)
    if new_pg == True:
        pg += 1




In [ ]:
#The polygon points are binned by latitude and frequency as a 2d array. 
#This counts the number of times a polygon was at a given frequency/latitude pair
ret = binned_statistic_2d(polygon_point_dict["GEO_LAT"], 
                          polygon_point_dict["Frequency"], 
                          None, 
                          'count', 
                          bins=[latitudes, frequencies], \
    expand_binnumbers=True)

print (ret.statistic)

print (ret.binnumber)

sums = np.zeros([len(latitudes), len(frequencies)])
#The counts are multiplied by the total flux measured at each frequency/latitude pair to give 
#the total flux at each point. 
for i in range(len(polygon_point_dict["GEO_LAT"])):
    m = ret.binnumber [0][i] - 1
    n = ret.binnumber [1][i] - 1
    sums[m][n] += polygon_point_dict["Flux"][i]


In [ ]:
np.shape(ret.binnumber), ret.binnumber

In [ ]:
np.shape(sums), np.shape(sums)[0]*np.shape(sums)[1]

In [ ]:
flux_min = np.quantile(sums[sums > 0.0], 0.05)
flux_max = np.quantile(sums[sums > 0.0], 0.95)
scaleZ = colors.LogNorm(vmin=flux_min, vmax=flux_max)

flux_min, flux_max

In [ ]:

fig, ax = pyplot.subplots()

im = plt.pcolormesh(latitudes, frequencies,  sums.T, norm = scaleZ, cmap='viridis')

fontsize=10
ax.set_xlabel('GEO Latitude',fontsize=fontsize)
ax.set_ylabel('Frequency (Hz)', fontsize=fontsize)
ax.set_yscale('log')

cb=fig.colorbar(im, ax=ax)
cbarlabel=r'Intensity (V$^2$/m$^2$/Hz)'
cb.set_label(cbarlabel, fontsize=fontsize)

plt.savefig(type_[0]+".png", dpi = 300, transparent = False)

fig.show()


In [ ]:
sns.histplot(polygon_point_dict["GEO_LAT"])
